In [111]:
from __future__ import division
import cPickle
import glob 
import json
import re

In [112]:
with open("../Data/corpus_sentences_py2.pk", 'rb') as f:
    corpus_sent = cPickle.load(f)
def getSetencesForCat(corpus, category):
    sentences={}
    for website, collection in corpus.items():
        relevant_setences = collection[category]
        sentences[website]=relevant_setences
    return sentences
relevantSents = getSetencesForCat(corpus_sent, 'User Access, Edit and Deletion')

In [113]:
scorefiles=glob.glob("runs/*_final_*/Scores*.json")

In [114]:
scorefiles

['runs/CNN_glove_final_FP_3151/Scores_FP.json',
 'runs/CNN_glove_final_DR_2998/Scores_DR.json',
 'runs/CNN_glove_final_DS_1795/Scores_DS.json',
 'runs/CNN_glove_final_DNT_3120/Scores_DNT.json',
 'runs/CNN_glove_final_UC_2206/Scores_UC.json',
 'runs/CNN_glove_final_UA_9271/Scores_UA.json']

In [ ]:
for scorefile in scorefiles:
    scoreresult=[]
    categoryScores=json.load(open(scorefile), "r")
    for category, score_dicts in categoryScores.iteritems():
        for website, scorelist in score_dicts.iteritems():
            scoresult.append({"site":website, "categories":[]})

In [69]:
test= [["To access the personally identifiable information that our Services collected about you online from the pages on which this Privacy Policy is posted, to correct factual errors in such information, or to update your interests, preferences or other portions of your user profile, please log into your profile if you have created one with us or send an email to privacy@flixster-inc.com.", 2], ["To protect your privacy and security, we will take reasonable steps to help verify your identity before granting access or making corrections.", 2]]

In [71]:
zip(*test)[1]

(2, 2)

In [115]:
def score_buffer(score_file_list):
    scoreresult={}
    for scorefile in score_file_list:
        categoryScores=json.load(open(scorefile, "r"))
        for category, score_dicts in categoryScores.items():
            for website, scorelist in score_dicts.items():
                scoreresult.setdefault(website, {})
                scoreresult[website].setdefault(category, {"good":[], "bad":[], "neutral":[]})
                if len(scorelist)>0:
                    scoreresult[website][category]["score"]=sum(zip(*scorelist)[1])/(len(scorelist)*2)
                    for sentence, score in scorelist:
                        if score ==0:
                            scoreresult[website][category]["bad"].append(sentence)
                        elif score ==1:
                            scoreresult[website][category]["neutral"].append(sentence)
                        elif score ==2:
                            scoreresult[website][category]["good"].append(sentence)
                else:
                    scoreresult[website][category]["score"]="Missing"
                    
    return scoreresult
intermediate=score_buffer(scorefiles)         

In [101]:
intermediate["popularmechanics.com"]["Data Retention"]

{'bad': [u'You can manage and delete flash cookies by visiting http://kb2.adobe.com/cps/526/52697ee8.html.',
  u'When we delete any information, it will be deleted from the active database, but may remain in our archives.'],
 'good': [u'You may request deletion of your Personally Identifiable Information by us, but please note that we may be required to keep this information and not delete it (or to keep this information for a certain time, in which case we will comply with your deletion request only after we have fulfilled such requirements).'],
 'neutral': [u'You may elect to change your User Registration at any time by logging in and editing your User Registration (each of our Covered Sites will contain log-in information).',
  u'If you cancel your User Registration, we will retain your email address, user name and assigned user ID on file and use them only for the purpose of assuring you are not automatically registered again, against your wishes.'],
 'score': 0.4}

In [116]:
def cleanSiteName(name):
    name=re.sub("www\\.", "", name)
    name=re.sub("\\.com", "", name)
    return name

In [120]:
def organizeJSON(intermediate_result):
    grades=["E", "D", "C", "B", "A"]
    finalresult=[]
    for website, information in intermediate_result.iteritems():
        website_result={}
        allscores=[]
        website_result["site"]=cleanSiteName(website)
        website_result.setdefault("categories",[])
        for category, info in information.iteritems():
            category_result=info
            category_result["category"]=category
            website_result["categories"].append(category_result)
            allscores.append(category_result["score"])
        validscores=[score for score in allscores if score !="Missing"]
        if len(validscores)>0:
            overallscore=sum(validscores)/len(validscores)*len(grades)-0.001
            finalgrade=grades[int(overallscore)]
        else:
            finalgrade="Missing"
        website_result["score"]=finalgrade
        finalresult.append(website_result)
    return finalresult
finalJSON = organizeJSON(intermediate)

In [121]:
with open("../Data/All_scores.json", "w") as f:
    json.dump(finalJSON, f)

## Model accuracies


In [130]:
import pandas as pd
performances=glob.glob("runs/*_final_*/*performance.json")
resultlist=[]
for result in performances:
    with open(result, "r") as f:
        resultlist.append(json.load(f))

resultTable=pd.DataFrame(resultlist)

simpletable=resultTable[["Category", "Precision", "Recall", "Development f1"]]

In [142]:
simpletable

,Category,Precision,Recall,Development f1
0,First Party Collection/Use,0.872639,0.892086,0.892086
1,Data Retention,0.792339,0.777778,0.777778
2,Data Security,0.760818,0.795276,0.795276
3,Do Not Track,0.604938,0.777778,0.777778
4,User Choice/Control,0.922787,0.955446,0.955446
5,"User Access, Edit and Deletion",0.871164,0.868852,0.868852


In [141]:
scorejson=simpletable.to_json("../Data/CNN_accuracies.json",orient="records")
